<a href="https://colab.research.google.com/github/francoriboratig/TP2-Datos/blob/master/Red%20convolucional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import numpy as np
import tensorflow as tf
import pandas as pd
import random
from google.colab import files

!pip install -q tensorflow-hub
!pip install -q tfds-nightly
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")


Version:  2.2.0
Eager mode:  True
Hub version:  0.8.0
GPU is NOT AVAILABLE


In [36]:
url_train = 'https://raw.githubusercontent.com/JulioCastillo1/TP1-Datos/master/train.csv'
url_test = 'https://raw.githubusercontent.com/francoriboratig/TP2-Datos/master/test.csv'

train_df = pd.read_csv(url_train)
test_df = pd.read_csv(url_test)
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [37]:
train_df = train_df.drop(['keyword', 'location'], axis = 1)

In [38]:
train_df.head()

,id,text,target
0,1,Our Deeds are the Reason of this #earthquake M...,1
1,4,Forest fire near La Ronge Sask. Canada,1
2,5,All residents asked to 'shelter in place' are ...,1
3,6,"13,000 people receive #wildfires evacuation or...",1
4,7,Just got sent this photo from Ruby #Alaska as ...,1


In [39]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"

hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)

train_text_list = np.asarray(train_df['text'])
train_target_list = np.asarray(train_df['target'])

hub_layer(train_text_list[:3])



<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 0.00723878,  0.15689617,  1.2953067 , -0.04077986, -0.16036902,
        -1.2004429 , -0.40451345, -0.5239726 , -0.23558664, -0.35022184,
        -0.28988177, -0.47197863, -0.07621863,  0.17048942, -0.8323773 ,
         0.02260084,  0.16125458, -0.64086777, -0.569074  ,  0.04000134],
       [-0.4983734 ,  1.2828879 , -0.18836124, -0.94734734,  0.8805808 ,
        -0.17885518, -0.01543166,  0.3832704 , -1.508848  ,  1.7532811 ,
        -0.75493276,  0.60124093, -0.14378418, -0.891944  ,  0.43213663,
         1.3467408 , -1.4859253 ,  1.5762157 , -0.37878054,  0.9423684 ],
       [ 0.08814443, -0.31140247,  0.6044149 ,  1.7130805 ,  1.3203121 ,
        -3.201303  ,  0.8445131 ,  0.9621795 , -1.5353138 ,  2.2069528 ,
        -0.2790253 ,  0.01095965, -2.2282734 ,  0.2043116 , -2.17501   ,
         1.3441613 ,  0.11884003, -1.2388006 , -0.25501075,  0.69527215]],
      dtype=float32)>

In [40]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_4 (KerasLayer)   (None, 20)                400020    
_________________________________________________________________
dense_6 (Dense)              (None, 16)                336       
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [41]:
model.compile(optimizer = 'adam',
              loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics = ['accuracy'])

# LO QUE SIGUE NO ANDA, no tengo train_data (ni validation_data, pero es análogo), deben
# tener formato Prefetch DataShape y no se cómo crearlo a partir de train_text_list, train_df o lo que sea

history = model.fit(train_text_list, train_target_list, batch_size=512,
                    epochs=20,
                    verbose=1)


Epoch 1/20
15/15 [==============================] - 0s 10ms/step - loss: 0.7003 - accuracy: 0.6280
Epoch 2/20
15/15 [==============================] - 0s 9ms/step - loss: 0.6110 - accuracy: 0.6796
Epoch 3/20
15/15 [==============================] - 0s 10ms/step - loss: 0.5673 - accuracy: 0.7092
Epoch 4/20
15/15 [==============================] - 0s 10ms/step - loss: 0.5397 - accuracy: 0.7288
Epoch 5/20
15/15 [==============================] - 0s 10ms/step - loss: 0.5208 - accuracy: 0.7406
Epoch 6/20
15/15 [==============================] - 0s 10ms/step - loss: 0.5059 - accuracy: 0.7507
Epoch 7/20
15/15 [==============================] - 0s 10ms/step - loss: 0.4938 - accuracy: 0.7566
Epoch 8/20
15/15 [==============================] - 0s 10ms/step - loss: 0.4827 - accuracy: 0.7637
Epoch 9/20
15/15 [==============================] - 0s 9ms/step - loss: 0.4722 - accuracy: 0.7708
Epoch 10/20
15/15 [==============================] - 0s 9ms/step - loss: 0.4625 - accuracy: 0.7759
Epoch 11/20


In [42]:
loss,precision = model.evaluate(train_text_list, train_target_list)
print('Precisión: %.2f' % (precision*100))

238/238 [==============================] - 0s 1ms/step - loss: 0.3632 - accuracy: 0.8380
Precisión: 83.80


In [43]:
test_df = test_df.drop(['keyword', 'location'], axis = 1)
test_df.head(5)

,id,text
0,0,Just happened a terrible car crash
1,2,"Heard about #earthquake is different cities, s..."
2,3,"there is a forest fire at spot pond, geese are..."
3,9,Apocalypse lighting. #Spokane #wildfires
4,11,Typhoon Soudelor kills 28 in China and Taiwan


In [44]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"

hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)

test_text_list = np.asarray(test_df['text'])

hub_layer(train_text_list[:3])

prediction=model.predict_classes(test_text_list)
prediction

array([[1],
       [1],
       [1],
       ...,
       [0],
       [1],
       [0]], dtype=int32)

In [45]:
test_df['target'] = prediction
export = pd.DataFrame()
export['id']=test_df['id']
export['target'] = test_df['target']
export= export.set_index('id')
export.to_csv('submit.csv')
files.download('submit.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>